In [ ]:
import caffe

In [ ]:
import tensorflow as tf

In [ ]:
'''
#Reference quora:
g = tf.Graph()
with g.as_default():
    tf_X = tf.convert_to_tensor(value=X, dtype=tf.float32)
    tf_y = tf.convert_to_tensor(value=y, dtype=tf.float32)
 
    if reuse_weights:
        weights_ = numpy.loadtxt(...)
        biases_ = numpy.loadtxt(...) 
        tf_weights_ = tf.Variable(weights_)
        tf_biases_ = tf.Variable(biases_)
    else: # initialize weights
        tf_weights_ = tf.Variable(tf.truncated_normal([...])
        tf_biases = tf.Variable(tf.zeros([...]))
    ...      
    train = optimizer.minimize(cost)
 
init = tf.initialize_all_variables()
with tf.Session(graph=g) as sess:
    sess.run(init)
    ...
    for e in epochs:
        _, c = sess.run([train, cost])
    ... 
    # when finished, save weights to numpy arrays
    weights_ = tf_weights_.eval()
    biases_ = tf_biases_.eval()
... 
 
# code save numpy arrays to CSV
numpy.savetxt(weights_, ...)
numpy.savetxt(biases_, ...)
'''

In [ ]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the 'License');
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an 'AS IS' BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""A simple MNIST classifier which displays summaries in TensorBoard.

This is an unimpressive MNIST model, but it is a good example of using
tf.name_scope to make a graph legible in the TensorBoard graph explorer, and of
naming summary tags so that they are grouped meaningfully in TensorBoard.

It demonstrates the functionality of every TensorBoard dashboard.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys

import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

FLAGS = None


def train():
  # Import data
  mnist = input_data.read_data_sets(FLAGS.data_dir,
                                    fake_data=FLAGS.fake_data)

  sess = tf.InteractiveSession()
  # Create a multilayer model.

  # Input placeholders
  with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x-input')
    y_ = tf.placeholder(tf.int64, [None], name='y-input')

  with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image('input', image_shaped_input, 10)

  # We can't initialize these variables to 0 - the network will get stuck.
  def weight_variable(shape):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

  def bias_variable(shape):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

  def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
      mean = tf.reduce_mean(var)
      tf.summary.scalar('mean', mean)
      with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
      tf.summary.scalar('stddev', stddev)
      tf.summary.scalar('max', tf.reduce_max(var))
      tf.summary.scalar('min', tf.reduce_min(var))
      tf.summary.histogram('histogram', var)

  def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.

    It does a matrix multiply, bias add, and then uses ReLU to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # Adding a name scope ensures logical grouping of the layers in the graph.
    with tf.name_scope(layer_name):
      # This Variable will hold the state of the weights for the layer
      with tf.name_scope('weights'):
        weights = weight_variable([input_dim, output_dim])
        variable_summaries(weights)
      with tf.name_scope('biases'):
        biases = bias_variable([output_dim])
        variable_summaries(biases)
      with tf.name_scope('Wx_plus_b'):
        preactivate = tf.matmul(input_tensor, weights) + biases
        tf.summary.histogram('pre_activations', preactivate)
      activations = act(preactivate, name='activation')
      tf.summary.histogram('activations', activations)
      return activations

  hidden1 = nn_layer(x, 784, 500, 'layer1')

  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    tf.summary.scalar('dropout_keep_probability', keep_prob)
    dropped = tf.nn.dropout(hidden1, keep_prob)

  # Do not apply softmax activation yet, see below.
  y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)

  with tf.name_scope('cross_entropy'):
    with tf.name_scope('total'):
      cross_entropy = tf.losses.sparse_softmax_cross_entropy(
          labels=y_, logits=y)
  tf.summary.scalar('cross_entropy', cross_entropy)

  with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(
        cross_entropy)

  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      correct_prediction = tf.equal(tf.argmax(y, 1), y_)
    with tf.name_scope('accuracy'):
      accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', accuracy)

  merged = tf.summary.merge_all()
  train_writer = tf.summary.FileWriter(FLAGS.log_dir + '/train', sess.graph)
  test_writer = tf.summary.FileWriter(FLAGS.log_dir + '/test')
  tf.global_variables_initializer().run()

  def feed_dict(train):
    """Make a TensorFlow feed_dict: maps data onto Tensor placeholders."""
    if train or FLAGS.fake_data:
      xs, ys = mnist.train.next_batch(100, fake_data=FLAGS.fake_data)
      k = FLAGS.dropout
    else:
      xs, ys = mnist.test.images, mnist.test.labels
      k = 1.0
    return {x: xs, y_: ys, keep_prob: k}

  for i in range(FLAGS.max_steps):
    if i % 10 == 0:  # Record summaries and test-set accuracy
      summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
      test_writer.add_summary(summary, i)
      print('Accuracy at step %s: %s' % (i, acc))
    else:  # Record train set summaries, and train
      if i % 100 == 99:  # Record execution stats
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        summary, _ = sess.run([merged, train_step],
                              feed_dict=feed_dict(True),
                              options=run_options,
                              run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        train_writer.add_summary(summary, i)
        print('Adding run metadata for', i)
      else:  # Record a summary
        summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
        train_writer.add_summary(summary, i)
  print(tf.contrib.graph_editor.get_tensors(tf.get_default_graph()))
  print (tf.get_default_graph().get_tensor_by_name('layer1/weights/truncated_normal/TruncatedNormal:0').eval())
  train_writer.close()
  test_writer.close()


def main(_):
  if tf.gfile.Exists(FLAGS.log_dir):
    tf.gfile.DeleteRecursively(FLAGS.log_dir)
  tf.gfile.MakeDirs(FLAGS.log_dir)
  train()




In [ ]:
if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  parser.add_argument('--fake_data', nargs='?', const=True, type=bool,
                      default=False,
                      help='If true, uses fake data for unit testing.')
  parser.add_argument('--max_steps', type=int, default=10,
                      help='Number of steps to run trainer.')
  parser.add_argument('--learning_rate', type=float, default=0.001,
                      help='Initial learning rate')
  parser.add_argument('--dropout', type=float, default=0.9,
                      help='Keep probability for training dropout.')
  parser.add_argument(
      '--data_dir',
      type=str,
      default=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                           'tensorflow/mnist/input_data'),
      help='Directory for storing input data')
  parser.add_argument(
      '--log_dir',
      type=str,
      default=os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                           'tensorflow/mnist/logs/mnist_with_summaries'),
      help='Summaries log directory')
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

In [ ]:
import facenet
import os
import tensorflow as tf

In [ ]:
def get_frozen_graph(model_dir = '/home/ovuser/FaceRecognition/models/20170512-110547'):
    with tf.Graph().as_default():
        with tf.Session() as sess:
            meta_file, ckpt_file = facenet.get_model_filenames(os.path.expanduser(model_dir))
            facenet.load_model(model_dir)
            all_tensors = tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
            lines = []
            with open('/home/ovuser/Desktop/CaffeFaceResearch/Facenet_Tensors', 'w') as fp:
                for tensor in all_tensors:
                    try:
                        lines.append(tensor.name + ' '+ ','.join([str(it) for it in tensor.get_shape()]) + '\n')
                    except ValueError:
                        print (tensor.name)
                fp.writelines(lines)
            graphdef = tf.get_default_graph().as_graph_def()
            op = sess.graph.get_operations()
            writer = tf.summary.FileWriter("/home/ovuser/research/logs/facenet")
            frozen_graph = tf.graph_util.convert_variables_to_constants(sess, graphdef, ["embeddings"])
            writer.add_graph(sess.graph)
            return tf.graph_util.remove_training_nodes(frozen_graph), sess.graph

In [ ]:
frozen_model = get_frozen_graph()

In [ ]:
print (len(frozen_model[0].node))

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Make sure that caffe is on the python path:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

# configure plotting
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [22]:
net = caffe.Net('/home/ovuser/Desktop/CaffeFaceResearch/facenet_deploy.prototxt', '/home/ovuser/Desktop/CaffeFaceResearch/face_model/facenet_inc_resnet_v1_softmax_512_iter_32000.caffemodel', caffe.TEST)
caffe_tensors = []
nodes = net.params.keys()
for node in nodes:
    for node_layer in net.params[node]:
        shape = node_layer.data.shape
        shape_str = ','.join([str(it) for it in shape])
        caffe_tensors.append(node+': '+shape_str+'\n')

In [23]:
with open('/home/ovuser/Desktop/CaffeFaceResearch/caffe_facenet_tensors.txt', 'w') as fp:
    fp.writelines(caffe_tensors)

In [24]:
def transfer_tf_net_params_to_numpy_arrays(model_dir = '/home/ovuser/FaceRecognition/models/20170512-110547'):
    with tf.Graph().as_default():
        with tf.Session() as sess:
            meta_file, ckpt_file = facenet.get_model_filenames(os.path.expanduser(model_dir))
            facenet.load_model(model_dir)
            all_tensors = tf.contrib.graph_editor.get_tensors(tf.get_default_graph())
            